In [1]:
#import libraliry to handle the url request
import urllib.request
!pip install bs4
!pip install lxml

In [26]:
#url for Canada List
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [27]:
page = urllib.request.urlopen(url)

In [28]:
#Import BeautifulSoup for handling with HTML & XML data
from bs4 import BeautifulSoup
soup = BeautifulSoup(page,'html.parser')

## Scraping table contents in WIKI

In [29]:
#Retrieve Table title
soup.title
soup.title.string

'List of postal codes of Canada: M - Wikipedia'

In [30]:
#scraping table contents
right_table=soup.find('table', class_='wikitable sortable')

A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
                               

## Create DataFrame and fit the table with df

In [31]:
#fit contents to DataFrame
import pandas as pd 
import numpy as np

df = pd.DataFrame()
df['Postal Code'] = A

df['Borough'] = B

df['Neighborhood'] = C
#Remove \n
df.replace('\n', '', regex=True, inplace=True)
df


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [32]:
#Filter out col[Borough] that without value
df = df[df['Borough']!= 'Not assigned']
df

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing Centre
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [33]:
#Check if any 'Not assigned' value in col[Neighborhood]

(df['Neighborhood']!= 'Not assigned').value_counts()

True    103
Name: Neighborhood, dtype: int64

In [34]:
#Check the DataFrame shape
no_row = df.shape[0]
print('Number of rows of the DataFrame is ', no_row)


Number of rows of the DataFrame is  103


In [35]:
#Read CSV file for geo location file
path = 'http://cocl.us/Geospatial_data'
cord = pd.read_csv(path)

In [36]:
#Join Table 
new_df = df.join(cord.set_index('Postal Code'), on='Postal Code')
new_df= new_df.reset_index()
new_df = new_df.drop('index',axis=1)
new_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [37]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(new_df['Borough'].unique()),
        new_df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [38]:
#Find out how many Unique Categories.
print('There are {} uniques categories.'.format(len(new_df['Neighborhood'].unique())))


There are 98 uniques categories.


## Analyze Each Neighborhood

In [39]:
# one hot encoding
toronto_onehot = pd.get_dummies(new_df[['Neighborhood']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = new_df['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])

toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

,Borough,Agincourt,"Alderwood, Long Branch","Bathurst Manor, Wilson Heights, Downsview North",Bayview Village,"Bedford Park, Lawrence Manor East",Berczy Park,"Birch Cliff, Cliffside West","Brockton, Parkdale Village, Exhibition Place",Business reply mail Processing Centre,...,"West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale",Westmount,Weston,"Wexford, Maryvale",Willowdale,"Willowdale, Newtonbrook",Woburn,Woodbine Heights,York Mills West,"York Mills, Silver Hills"
0,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,Etobicoke,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,East Toronto,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
101,Etobicoke,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
toronto_onehot.shape

(103, 99)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [41]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
#toronto_grouped.reset_index()
toronto_grouped

,Borough,Agincourt,"Alderwood, Long Branch","Bathurst Manor, Wilson Heights, Downsview North",Bayview Village,"Bedford Park, Lawrence Manor East",Berczy Park,"Birch Cliff, Cliffside West","Brockton, Parkdale Village, Exhibition Place",Business reply mail Processing Centre,...,"West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale",Westmount,Weston,"Wexford, Maryvale",Willowdale,"Willowdale, Newtonbrook",Woburn,Woodbine Heights,York Mills West,"York Mills, Silver Hills"
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,Downtown Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000
4,Etobicoke,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.083333,0.083333,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,North York,0.000000,0.000000,0.041667,0.041667,0.041667,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.083333,0.041667,0.000000,0.0,0.041667,0.041667
7,Scarborough,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.0,...,0.000000,0.000000,0.0,0.058824,0.000000,0.000000,0.058824,0.0,0.000000,0.000000
8,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
9,York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [42]:
toronto_grouped.shape

(10, 99)

In [43]:
num_top_venues = 5

for hood in toronto_grouped['Borough']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
                                               venue  freq
0                        Moore Park, Summerhill East  0.11
1                           Forest Hill North & West  0.11
2                                      Lawrence Park  0.11
3                                 North Toronto West  0.11
4  Summerhill West, Rathnelly, South Hill, Forest...  0.11


----Downtown Toronto----
                                    venue  freq
0               Regent Park, Harbourfront  0.05
1                                Christie  0.05
2                Garden District, Ryerson  0.05
3  First Canadian Place, Underground city  0.05
4                          Stn A PO Boxes  0.05


----East Toronto----
                                   venue  freq
0                        Studio District   0.2
1  Business reply mail Processing Centre   0.2
2                            The Beaches   0.2
3         India Bazaar, The Beaches West   0.2
4           The Danforth West, Riverdale   0.2




In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Lawrence Park,Roselawn,Davisville North,Davisville,"Moore Park, Summerhill East",Forest Hill North & West,North Toronto West,"Summerhill West, Rathnelly, South Hill, Forest...","The Annex, North Midtown, Yorkville","York Mills, Silver Hills"
1,Downtown Toronto,"Kensington Market, Chinatown, Grange Park","Toronto Dominion Centre, Design Exchange","Richmond, Adelaide, King",Rosedale,"First Canadian Place, Underground city",St. James Town,"St. James Town, Cabbagetown","Commerce Court, Victoria Hotel",Stn A PO Boxes,"Garden District, Ryerson"
2,East Toronto,"India Bazaar, The Beaches West",Studio District,Business reply mail Processing Centre,The Beaches,"The Danforth West, Riverdale","York Mills, Silver Hills",Glencairn,"Dufferin, Dovercourt Village",East Toronto,"Eringate, Bloordale Gardens, Old Burnhamthorpe..."
3,East York,Leaside,Thorncliffe Park,Woodbine Heights,East Toronto,"Parkview Hill, Woodbine Gardens","Dufferin, Dovercourt Village","Eringate, Bloordale Gardens, Old Burnhamthorpe...","Fairview, Henry Farm, Oriole","First Canadian Place, Underground city",Forest Hill North & West
4,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",Westmount,"The Kingsway, Montgomery Road, Old Mill North","New Toronto, Mimico South, Humber Bay Shores","Mimico NW, The Queensway West, South of Bloor,...","Old Mill South, King's Mill Park, Sunnylea, Hu...","West Deane Park, Princess Gardens, Martin Grov...","South Steeles, Silverstone, Humbergate, Jamest...","Eringate, Bloordale Gardens, Old Burnhamthorpe...",Islington Avenue
5,Mississauga,Canada Post Gateway Processing Centre,"High Park, The Junction South",East Toronto,"Eringate, Bloordale Gardens, Old Burnhamthorpe...","Fairview, Henry Farm, Oriole","First Canadian Place, Underground city",Forest Hill North & West,"Garden District, Ryerson",Glencairn,"Golden Mile, Clairlea, Oakridge"
6,North York,Downsview,Don Mills,Willowdale,"Lawrence Manor, Lawrence Heights",York Mills West,"Humberlea, Emery",Humber Summit,Hillcrest Village,"North Park, Maple Leaf Park, Upwood Park","Northwood Park, York University"
7,Scarborough,Agincourt,Cedarbrae,"Malvern, Rouge","Milliken, Agincourt North, Steeles East, L'Amo...","Guildwood, Morningside, West Hill","Golden Mile, Clairlea, Oakridge","Dorset Park, Wexford Heights, Scarborough Town...","Rouge Hill, Port Union, Highland Creek","Steeles West, L'Amoreaux West","Cliffside, Cliffcrest, Scarborough Village West"
8,West Toronto,"High Park, The Junction South","Parkdale, Roncesvalles","Little Portugal, Trinity","Runnymede, Swansea","Brockton, Parkdale Village, Exhibition Place","Dufferin, Dovercourt Village","Eringate, Bloordale Gardens, Old Burnhamthorpe...","Fairview, Henry Farm, Oriole","First Canadian Place, Underground city",Forest Hill North & West
9,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Weston,Humewood-Cedarvale,Caledonia-Fairbanks,"Runnymede, The Junction North","York Mills, Silver Hills","Guildwood, Morningside, West Hill",East Toronto,"Eringate, Bloordale Gardens, Old Burnhamthorpe...","Fairview, Henry Farm, Oriole"


In [68]:
!conda install -c conda-forge geopy --yes 
# uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

# All requested packages already installed.



In [69]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


## Cluster Neighborhoods

In [70]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



In [71]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([7, 9, 4, 2, 8, 0, 1, 6, 5, 3], dtype=int32)

In [52]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = new_df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Borough'), on='Borough')

toronto_merged


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Downsview,Don Mills,Willowdale,"Lawrence Manor, Lawrence Heights",York Mills West,"Humberlea, Emery",Humber Summit,Hillcrest Village,"North Park, Maple Leaf Park, Upwood Park","Northwood Park, York University"
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Downsview,Don Mills,Willowdale,"Lawrence Manor, Lawrence Heights",York Mills West,"Humberlea, Emery",Humber Summit,Hillcrest Village,"North Park, Maple Leaf Park, Upwood Park","Northwood Park, York University"
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,9,"Kensington Market, Chinatown, Grange Park","Toronto Dominion Centre, Design Exchange","Richmond, Adelaide, King",Rosedale,"First Canadian Place, Underground city",St. James Town,"St. James Town, Cabbagetown","Commerce Court, Victoria Hotel",Stn A PO Boxes,"Garden District, Ryerson"
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Downsview,Don Mills,Willowdale,"Lawrence Manor, Lawrence Heights",York Mills West,"Humberlea, Emery",Humber Summit,Hillcrest Village,"North Park, Maple Leaf Park, Upwood Park","Northwood Park, York University"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,9,"Kensington Market, Chinatown, Grange Park","Toronto Dominion Centre, Design Exchange","Richmond, Adelaide, King",Rosedale,"First Canadian Place, Underground city",St. James Town,"St. James Town, Cabbagetown","Commerce Court, Victoria Hotel",Stn A PO Boxes,"Garden District, Ryerson"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,8,"Kingsview Village, St. Phillips, Martin Grove ...",Westmount,"The Kingsway, Montgomery Road, Old Mill North","New Toronto, Mimico South, Humber Bay Shores","Mimico NW, The Queensway West, South of Bloor,...","Old Mill South, King's Mill Park, Sunnylea, Hu...","West Deane Park, Princess Gardens, Martin Grov...","South Steeles, Silverstone, Humbergate, Jamest...","Eringate, Bloordale Gardens, Old Burnhamthorpe...",Islington Avenue
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,9,"Kensington Market, Chinatown, Grange Park","Toronto Dominion Centre, Design Exchange","Richmond, Adelaide, King",Rosedale,"First Canadian Place, Underground city",St. James Town,"St. James Town, Cabbagetown","Commerce Court, Victoria Hotel",Stn A PO Boxes,"Garden District, Ryerson"
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558,4,"India Bazaar, The Beaches West",Studio District,Business reply mail Processing Centre,The Beaches,"The Danforth West, Riverdale","York Mills, Silver Hills",Glencairn,"Dufferin, Dovercourt Village",East Toronto,"Eringate, Bloordale Gardens, Old Burnhamthorpe..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,8,"Kingsview Village, St. Phillips, Martin Grove ...",Westmount,"The Kingsway, Montgomery Road, Old Mill North","New Toronto, Mimico South, Humber Bay Shores","Mimico NW, The Queensway West, South of Bloor,...","Old Mill South, King's Mill Park, Sunnylea, Hu...","West Deane Park, Princess Gardens, Martin Grov...","South Steeles, Silverstone, Humbergate, Jamest...","Eringate, Bloordale Gardens, Old Burnhamthorpe...",Islington Avenue


In [54]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

### Cluster 0

In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head(1)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
76,Mississauga,0,Canada Post Gateway Processing Centre,"High Park, The Junction South",East Toronto,"Eringate, Bloordale Gardens, Old Burnhamthorpe...","Fairview, Henry Farm, Oriole","First Canadian Place, Underground city",Forest Hill North & West,"Garden District, Ryerson",Glencairn,"Golden Mile, Clairlea, Oakridge"


### Cluster 1

In [65]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head(1)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Downsview,Don Mills,Willowdale,"Lawrence Manor, Lawrence Heights",York Mills West,"Humberlea, Emery",Humber Summit,Hillcrest Village,"North Park, Maple Leaf Park, Upwood Park","Northwood Park, York University"


### Cluster 2

In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head(1)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,East York,2,Leaside,Thorncliffe Park,Woodbine Heights,East Toronto,"Parkview Hill, Woodbine Gardens","Dufferin, Dovercourt Village","Eringate, Bloordale Gardens, Old Burnhamthorpe...","Fairview, Henry Farm, Oriole","First Canadian Place, Underground city",Forest Hill North & West
